In [115]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [116]:
from argparse import Namespace
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
import numpy as np
import pandas as pd
import os

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [117]:
!pip install konlpy
from konlpy.tag import Kkma
kkma = Kkma()

In [118]:
args = Namespace(
    source_data_path="/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter8_Seq_Modeling_Advanced/data/nmt/eng-kor.txt",
    output_data_path="/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter8_Seq_Modeling_Advanced/data/nmt/mytest_eng_kor.csv",
    perc_train=0.7,
    perc_val=0.15,
    perc_test=0.15,
    seed=1337
)
assert args.perc_test > 0 and (args.perc_test + args.perc_val + args.perc_train == 1.0)

In [119]:
with open(args.source_data_path) as fp:
  lines = fp.readlines()

lines = [line.replace("\n", "").lower().split("\t") for line in lines]

In [120]:
for line in lines[1000:1006]:
  print(line)
print(len(lines))

["i'm embarrassed.", '창피해.', 'cc-by 2.0 (france) attribution: tatoeba.org #1217164 (wallebot) & #8365860 (eunhee)']
["i'm heartbroken.", '제 마음이 아파요.', 'cc-by 2.0 (france) attribution: tatoeba.org #3824090 (ck) & #6305028 (queserasera)']
["i'm just sleepy.", '나 졸려.', 'cc-by 2.0 (france) attribution: tatoeba.org #9732840 (ck) & #11104818 (intertime)']
["i'm not sulking.", '나 삐친 거 아니야.', 'cc-by 2.0 (france) attribution: tatoeba.org #2248186 (ck) & #8386503 (eunhee)']
["i'm on the list.", '나는 명단에 있다.', 'cc-by 2.0 (france) attribution: tatoeba.org #1891097 (ck) & #11106751 (intertime)']
['is tom with you?', '톰이랑 같이 있어?', 'cc-by 2.0 (france) attribution: tatoeba.org #1886872 (ck) & #10163048 (eunhee)']
5892


In [121]:
"""
import pandas as pd
import urllib.request
from tokenizers import BertWordPieceTokenizer

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)

train_txt_korean = []
for line in lines:
  train_txt_korean.append(line[1])

print(len(train_txt_korean))

base_path = "/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter8_Seq_Modeling_Advanced/data/nmt/"
with open(os.path.join(base_path, "kor-tokenize-train.txt"), "w") as fp:
  for text in train_txt_korean:
    fp.write(text +"\n")

data_file = os.path.join(base_path, "kor-tokenize-train.txt")
vocab_size = 30000
limit_alphabet = 6000
min_frequency = 5

tokenizer.train(files=data_file,
                vocab_size=vocab_size,
                limit_alphabet=limit_alphabet,
                min_frequency=min_frequency)

# vocab 저장
#tokenizer.save_model('"/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter8_Seq_Modeling_Advanced/')
"""

'\nimport pandas as pd\nimport urllib.request\nfrom tokenizers import BertWordPieceTokenizer\n\nurllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")\ntokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)\n\ntrain_txt_korean = []\nfor line in lines:\n  train_txt_korean.append(line[1])\n\nprint(len(train_txt_korean))\n\nbase_path = "/content/drive/MyDrive/Github_NLP/NLP_with_PyTorch/Chapter8_Seq_Modeling_Advanced/data/nmt/"\nwith open(os.path.join(base_path, "kor-tokenize-train.txt"), "w") as fp:\n  for text in train_txt_korean:\n    fp.write(text +"\n")\n\ndata_file = os.path.join(base_path, "kor-tokenize-train.txt")\nvocab_size = 30000\nlimit_alphabet = 6000\nmin_frequency = 5\n\ntokenizer.train(files=data_file,\n                vocab_size=vocab_size,\n                limit_alphabet=limit_alphabet,\n                min_frequency=min_frequency)\n\n# vocab 저장\n#tokenizer.save_model(\'"/content/drive/MyDrive

In [126]:
data = []
for english_sentence, korean_sentence, _ in lines:
  data.append({
      "english_tokens": word_tokenize(english_sentence, language="english"),
      "korean_tokens" : kkma.morphs(korean_sentence)
  })


In [127]:
print(len(data))

5892


In [128]:
for l in data[3000:3050]:
  print(l)

{'english_tokens': ['tom', 'hates', 'going', 'to', 'school', '.'], 'korean_tokens': ['톰', '은', '학교', '에', '가', '기', '싫', '다', '.']}
{'english_tokens': ['tom', 'is', 'a', 'pretty', 'good', 'poet', '.'], 'korean_tokens': ['톰', '은', '꽤', '괜찮', '은', '시인', '이', '야', '.']}
{'english_tokens': ['tom', 'is', 'a', 'pretty', 'good', 'poet', '.'], 'korean_tokens': ['톰', '은', '좀', '괜찮', '은', '시인', '이', '야', '.']}
{'english_tokens': ['tom', 'is', 'addicted', 'to', 'heroin', '.'], 'korean_tokens': ['톰', '은', '헤로인', '중독자', '야', '.']}
{'english_tokens': ['tom', 'is', 'having', 'a', 'hard', 'time', '.'], 'korean_tokens': ['톰', '은', '힘들', 'ㄴ', '시간', '을', '보내', '고', '있', '어', '.']}
{'english_tokens': ['tom', 'is', 'lonely', 'and', 'unhappy', '.'], 'korean_tokens': ['톰', '은', '외롭', '고', '불행', '하', '어', '.']}
{'english_tokens': ['tom', 'is', 'reading', 'a', 'magazine', '.'], 'korean_tokens': ['톰', '은', '잡지', '를', '읽', '고', '있', '어', '.']}
{'english_tokens': ['tom', 'is', 'shouting', 'something', '.'], 'kore

In [129]:
np.random.seed(args.seed)
np.random.shuffle(data)

In [130]:
for l in data[:5]:
  print(l)

{'english_tokens': ['they', 'refused', '.'], 'korean_tokens': ['그', '사람', '들', '은', '거절', '하', '었', '어', '.']}
{'english_tokens': ['machines', 'that', 'his', 'company', 'produces', 'are', 'superior', 'to', 'ours', '.'], 'korean_tokens': ['그', '의', '회사', '가', '생산', '하', '는', '기계', '가', '우리', '것', '보다', '뛰어나', '다', '.']}
{'english_tokens': ['i', 'understood', '.'], 'korean_tokens': ['이해', '하', '었', '어', '.']}
{'english_tokens': ['tom', 'was', 'diagnosed', 'with', 'autism', 'and', 'severe', 'depression', '.'], 'korean_tokens': ['톰', '은', '자폐증', '과', '심각', '하', 'ㄴ', '우울증', '으로', '진단', '받', '았', '어', '.']}
{'english_tokens': ['tom', 'will', 'be', 'in', 'his', 'room', 'all', 'day', 'today', '.'], 'korean_tokens': ['톰', '은', '오늘', '하루', '종일', '방', '안', '에', '있', '을', '것', '이', '다', '.']}


In [131]:
for datum in data:
  datum["source_language"] = " ".join(datum.pop("english_tokens"))
  datum["target_language"] = " ".join(datum.pop("korean_tokens"))


In [132]:
n_train = int(len(data) * args.perc_train)
n_val = int(len(data) * args.perc_val)

for datum in data[:n_train]:
    datum['split'] = 'train'

for datum in data[n_train:n_train+n_val]:
    datum['split'] = 'val'

for datum in data[n_train+n_val:]:
    datum['split'] = 'test'

In [133]:
for dict1 in data:
  print(dict1)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
{'source_language': 'this sounds reasonable .', 'target_language': '합리적 으로 들리 네 .', 'split': 'train'}
{'source_language': 'tom said boston is a nice place to visit .', 'target_language': '톰 은 보스턴 이 가보 기 좋 은 곳 이 라고 하 었 다 .', 'split': 'train'}
{'source_language': 'tom is a common name .', 'target_language': '톰 은 흔하 ㄴ 이름 이 다 .', 'split': 'train'}
{'source_language': "it was n't tom who talked to me .", 'target_language': '나오 아 얘기 하 ㄴ 사람 은 톰 이 아니 었 다 .', 'split': 'train'}
{'source_language': 'he laughed .', 'target_language': '그 는 웃 었 다 .', 'split': 'train'}
{'source_language': 'his sudden appearance surprised us all .', 'target_language': '걔 가 갑자기 오 아서 우리 는 다 놀라 었 어', 'split': 'train'}
{'source_language': 'i like learning languages .', 'target_language': '낳 ㄴ 언어 를 배우 는 것 을 좋아하 어 .', 'split': 'train'}
{'source_language': 'someone knocked on the door .', 'target_language': '누군가 문 을 똑똑 두드리 었 다 .', 'split': 'train'}
{'source_language': "i 'm not sure what i

In [134]:
nmt_df = pd.DataFrame(data)

In [135]:
nmt_df.groupby("split").count()

,source_language,target_language
split,,
test,885,885
train,4124,4124
val,883,883


In [136]:
nmt_df.loc[nmt_df.split == "train"].head()

,source_language,target_language,split
0,they refused .,그 사람 들 은 거절 하 었 어 .,train
1,machines that his company produces are superio...,그 의 회사 가 생산 하 는 기계 가 우리 것 보다 뛰어나 다 .,train
2,i understood .,이해 하 었 어 .,train
3,tom was diagnosed with autism and severe depre...,톰 은 자폐증 과 심각 하 ㄴ 우울증 으로 진단 받 았 어 .,train
4,tom will be in his room all day today .,톰 은 오늘 하루 종일 방 안 에 있 을 것 이 다 .,train


In [137]:
nmt_df.loc[nmt_df.split == "val"].head()


,source_language,target_language,split
4124,follow me .,따라오 아 .,val
4125,i did n't know that you loved tom so much .,네가 톰 을 그렇 게 사랑 하 ㄹ 줄 은 모르 았 어 .,val
4126,can i come some other time ?,나중 에 한번 다시 오 아도 되 어 ?,val
4127,why do you want a car ?,왜 자동차 를 갖 고 싶 어 ?,val
4128,tom will explain this to you later .,톰 이 나중 에 너 한테 설명 하 ㄹ 거 야 .,val


In [138]:
nmt_df.loc[nmt_df.split == "test"].head()

,source_language,target_language,split
5007,how strange !,참 이상 하 네 !,test
5008,forget it .,잊어버리 어 .,test
5009,i wonder what the weather will be tomorrow .,내일 날씨 가 어떻 ㄹ지 궁금 하 어 .,test
5010,the students of this school are kind .,이 학교 의 학생 들 은 친절 하 다 .,test
5011,how dare you !,어떻 게 감히 그러 ㄹ 수가 !,test


In [139]:
nmt_df.to_csv(args.output_data_path)